In [3]:
import shapefile
from pyproj import Proj, transform
import os, time

#apply projection to all shapefiles
lcp_path = '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles'

for year in sorted(os.listdir(lcp_path)):
    if year[0] == '.':
        continue
    print(year,'*********************************************************************')
    wildfire_dir = lcp_path+'/'+year
    for wildfire_name in sorted(os.listdir(wildfire_dir)):
        if wildfire_name[0] == '.':
            continue
        elif(wildfire_name.split('_')[0] == 'ak' or wildfire_name.split('_')[0] == 'AK'):
            continue
        elif(wildfire_name.split('_')[0] == 'hi'):
            continue
        else:
            print(wildfire_name)
            
            #project shapefiles from EPSG:4326 to EPSG:5070
            shp_dir = wildfire_name    #first folder of the wildfire_name's directory
            print("shp_dir = ", shp_dir)
            shp_dir_path = wildfire_dir+'/'+wildfire_name
            print("shp_dir_path = ", shp_dir_path)
            shp_file_path = shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+sorted(os.listdir(shp_dir_path))[0]+'.shp'
            print("shp_file_path = ", shp_file_path)
            out_shp_file_path = shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+'proj'+sorted(os.listdir(shp_dir_path))[0]+'.shp'
            print("out_shp_file_path = ", out_shp_file_path)
            
            try:
                os.system("rm "+shp_dir_path+'/'+"proj*")
                os.system("rm "+shp_dir_path+'/'+sorted(os.listdir(shp_dir_path))[0]+'/'+'proj*')                
            except:
                continue
            
            #pass the dirs containing proj~.* files
            if not os.path.isfile(out_shp_file_path):
                shpf = shapefile.Reader(shp_file_path)    #read shapefile
                wgs_shp = shapefile.Writer(out_shp_file_path, shapefile.POLYGON)    #write shapefile

                fields = shpf.fields
                wgs_fields = wgs_shp.fields

                for name in fields:
                    if type(name) == "tuple":
                        continue
                    else:
                        args = name
                        wgs_shp.field(*args)    #write the source shapefile's fields to the new shapefile

                records = shpf.records()        #record the source shapefile

                for row in records:
                    args = row    
                    wgs_shp.record(*args)       #write the record to the new shapefile

                input_projection = Proj("epsg:4326")
                output_projection = Proj("epsg:5070")

                geom = shpf.shapes()

                #geometry features loop
                for feature in geom:
                    # if there is only one part
                    if len(feature.parts) == 1:
                        # create empty list to store all the coordinates
                        poly_list = []
                        # get each coord that makes up the polygon
                        for coords in feature.points:            
                            x, y = coords[0], coords[1]
                            # tranform the coord
                            new_x, new_y = transform(input_projection, output_projection, y, x)
                            # put the coord into a list structure
                            poly_coord = [float(new_x), float(new_y)]
                            # append the coords to the polygon list
                            poly_list.append(poly_coord)
                        # add the geometry to the shapefile.
                        wgs_shp.poly([poly_list])
                    else:
                        # append the total amount of points to the end of the parts list
                        feature.parts.append(len(feature.points))
                        # enpty list to store all the parts that make up the complete feature
                        poly_list = []
                        # keep track of the part being added
                        parts_counter = 0

                        # while the parts_counter is less than the amount of parts
                        while parts_counter < len(feature.parts) - 1:
                            # keep track of the amount of points added to the feature
                            coord_count = feature.parts[parts_counter]
                            # number of points in each part
                            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
                            # create list to hold individual parts - these get added to poly_list[]
                            part_list = []
                            # cut off point for each part
                            end_point = coord_count + no_of_points

                            # loop through each part
                            while coord_count < end_point:
                                for coords in feature.points[coord_count:end_point]:
                                    x, y = coords[0], coords[1]
                                    # tranform the coord
                                    new_x, new_y = transform(input_projection, output_projection, y, x)
                                    # put the coord into a list structure
                                    poly_coord = [float(new_x), float(new_y)]
                                    # append the coords to the part list
                                    part_list.append(poly_coord)
                                    coord_count = coord_count + 1
                            # append the part to the poly_list
                            poly_list.append(part_list)
                            parts_counter = parts_counter + 1 
                        # add the geometry to to new file 
                        wgs_shp.poly(poly_list)

                wgs_shp.close() #automatically save shapefile

                #write the projection file
                prj = open(out_shp_file_path.split('.')[0]+".prj", "w")            
                epsg_str = 'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]'
                prj.write(epsg_str)
                prj.close()
                print()

print("Shapefile projection complete!!")

2016 *********************************************************************
al_caney_head
shp_dir =  al_caney_head
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head/20160321_0000/20160321_0000.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head/20160321_0000/proj20160321_0000.shp


rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_caney_head/proj*': No such file or directory
/tmp/ipykernel_5946/1534746164.py:75: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  new_x, new_y = transform(input_projection, output_projection, y, x)



al_half_way
shp_dir =  al_half_way
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_half_way
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_half_way/20161112_1417/20161112_1417.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_half_way/20161112_1417/proj20161112_1417.shp

al_lake_chinnabee
shp_dir =  al_lake_chinnabee
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lake_chinnabee
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lake_chinnabee/20161128_0000/20161128_0000.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lake_chinnabee/20161128_0000/proj20161128_0000.shp

al_lookout_mountain
shp_dir =  al_lookout_mountain
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lookout_mountain
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/Wildf

rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_half_way/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lake_chinnabee/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_lookout_mountain/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/al_power_horn/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/ar_pipeline/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/ar_sweetwater/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/ar_whitaker_point/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_air


az_lion_fta
shp_dir =  az_lion_fta
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_lion_fta
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_lion_fta/20160513_0000/20160513_0000.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_lion_fta/20160513_0000/proj20160513_0000.shp

az_maple
shp_dir =  az_maple
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_maple
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_maple/20160622_0000/20160622_0000.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_maple/20160622_0000/proj20160622_0000.shp

az_mormon
shp_dir =  az_mormon
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_mormon
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2016/az_mormon/20160524_2014/20160524_2014.shp
out_shp_file_p

...


or_round_butte
shp_dir =  or_round_butte
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_round_butte
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_round_butte/20190722_1200/20190722_1200.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_round_butte/20190722_1200/proj20190722_1200.shp

or_sexton_mt_lookout
shp_dir =  or_sexton_mt_lookout
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_sexton_mt_lookout
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_sexton_mt_lookout/20190921_1740/20190921_1740.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_sexton_mt_lookout/20190921_1740/proj20190921_1740.shp

or_ward
shp_dir =  or_ward
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_ward
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireSha

rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_round_butte/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_sexton_mt_lookout/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/or_ward/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/ut_alaska/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/ut_chippean/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/ut_green_ravine/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/ut_gun_range/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/ut_highway_101/proj


wa_left_hand
shp_dir =  wa_left_hand
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_left_hand
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_left_hand/20190724_1136/20190724_1136.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_left_hand/20190724_1136/proj20190724_1136.shp

wa_mt._dana
shp_dir =  wa_mt._dana
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt._dana
shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt._dana/20190901_0000/20190901_0000.shp
out_shp_file_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt._dana/20190901_0000/proj20190901_0000.shp

wa_mt.prj
shp_dir =  wa_mt.prj
shp_dir_path =  /home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt.prj


rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_left_hand/proj*': No such file or directory
rm: cannot remove '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt._dana/proj*': No such file or directory


NotADirectoryError: [Errno 20] Not a directory: '/home/qweddww123/farsite/TrainDataGen/WildfireShapefiles/2019/wa_mt.prj'